In [1]:
import numpy as np
import pandas as pd
import os
import re
import emoji

from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize



In [2]:

df=pd.read_csv('nlptrain.csv')
df.head()

,textID,text,selected_text,sentiment
0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative
2,088c60f138,my boss is bullying me...,bullying me,negative
3,9642c003ef,what interview! leave me alone,leave me alone,negative
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative


# Missing Values

In [3]:
def missingValues(data):
    total=data.isnull().sum().sort_values(ascending=False)
    percentage=(total/len(data))
    return pd.concat([total,percentage],axis=1,keys=['Total','Percentage'])

missingValues(df)

,Total,Percentage
selected_text,1,0.000036
text,1,0.000036
sentiment,0,0.000000
textID,0,0.000000


In [4]:
df=df.dropna()

In [5]:
def countValues(df,col):
    counts=df[col].value_counts().dropna()
    percentage=round(counts*100/np.sum(counts.values),2)
    return pd.concat([counts,percentage],axis=1,keys=['Total','Percentage'])
    
countValues(df,'sentiment')

,Total,Percentage
neutral,11117,40.45
positive,8582,31.23
negative,7781,28.32


In [6]:
def uniqueValues(df,col):
    uniques=np.unique(df[col],return_counts=True)
    return pd.DataFrame({'Title':uniques[0],
                         'Counts':uniques[1]})


uniqueValues(df,'sentiment')

,Title,Counts
0,negative,7781
1,neutral,11117
2,positive,8582


In [7]:
def duplicateValues(df):
    dups=[]
    columns=df.columns
    for i in df.columns:
        dups.append(sum(df[i].duplicated()))
    return pd.concat([pd.Series(columns),pd.Series(dups)],axis=1,
                    keys=['Columns','Counts'])
duplicateValues(df)

,Columns,Counts
0,textID,0
1,text,0
2,selected_text,5017
3,sentiment,27477


In [8]:
df.describe()

,textID,text,selected_text,sentiment
count,27480,27480,27480,27480
unique,27480,27480,22463,3
top,1a09cbe62d,"awww, we JUST missed you then!! But I`ll see...",good,neutral
freq,1,1,199,11117


In [9]:
def find_url(string):
    text=re.findall('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+',string)
    return ''.join(text)
    
    
find_url('I love spending time at https://www.kaggle.com/')

'https://www.kaggle.com/'

In [10]:
df['url']=df['text'].apply(lambda x: find_url(x))

In [11]:
df.head(10)['url']

0                                  
1                                  
2                                  
3                                  
4                                  
5    http://www.dothebouncy.com/smf
6                                  
7                                  
8                                  
9                                  
Name: url, dtype: object

In [12]:
def removeEmoji(text):
    text=text.split()
    text=[i for i in text if i.isalpha()]
    text=' '.join(text)
    return text

In [13]:
removeEmoji("I love ⚽ very much 😁")


'I love very much'

In [14]:
sentence="Its all about \U0001F600 face"
removeEmoji(sentence)

'Its all about face'

In [15]:
def findEmail(text):
    line=re.findall(r'[\w\.-]+@[\w\.-]+',text)
    return ','.join(line)
    
    

findEmail("My gmail is abc99@gmail.com")

'abc99@gmail.com'

In [16]:
df['email']=df['text'].apply(lambda x: findEmail(x))

In [23]:
(df['email']=='').value_counts()

True    27480
Name: email, dtype: int64

In [24]:
def findHash(text):
    line=re.findall(r'(?))

,textID,text,selected_text,sentiment,url,email
0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral,,
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative,,
2,088c60f138,my boss is bullying me...,bullying me,negative,,
3,9642c003ef,what interview! leave me alone,leave me alone,negative,,
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative,,
5,28b57f3990,http://www.dothebouncy.com/smf - some shameles...,http://www.dothebouncy.com/smf - some shameles...,neutral,http://www.dothebouncy.com/smf,
6,6e0c6d75b1,2am feedings for the baby are fun when he is a...,fun,positive,,
7,50e14c0bb8,Soooo high,Soooo high,neutral,,
8,e050245fbd,Both of you,Both of you,neutral,,
9,fc2cbefa9d,Journey!? Wow... u just became cooler. hehe....,Wow... u just became cooler.,positive,,
